In [5]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from io import StringIO
import requests
import unicodedata
import zipfile
from dbfread import DBF

# Função para determinar o delimitador do arquivo CSV
def determinar_delimitador(content):
    delimiters = [',', ';', '\t']
    for delimiter in delimiters:
        if delimiter in content:
            return delimiter
    return ','  # Delimitador padrão é a vírgula

def remove_special_characters(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

def normalize_and_uppercase(text):
    return remove_special_characters(str(text)).upper()

def download_csv_and_show_table():
    source_url = source_entry.get()
    col_index = col_index_entry.get()

    csv_stream = download_csv(source_url)

    if csv_stream:
        # Agora, antes de ler o CSV, determinamos o delimitador correto
        content = csv_stream.getvalue()
        delimiter = determinar_delimitador(content)

        try:
            df = pd.read_csv(csv_stream, delimiter=delimiter, on_bad_lines='skip')
        except pd.errors.ParserError as e:
            print(f"Erro ao ler o CSV: {e}")
            return

        if col_index:
            col_index = int(col_index)
            if col_index < len(df.columns):
                df = df.iloc[:, [col_index]]
            else:
                print("Índice de coluna fora do intervalo")
                return
        else:
            df = df.iloc[:, :4]

        for col in df.columns:
            df[col] = df[col].apply(normalize_and_uppercase)

        # Passo 1: Baixe e descompacte o arquivo ZIP
        with requests.get(source_url, stream=True) as response:
            if response.status_code == 200:
                with zipfile.ZipFile(response.content) as zip_ref:
                    zip_ref.extractall("temp_unzip_dir")

        # Passo 2: Acesse o arquivo .dbf no diretório descompactado
        dbf_file_path = "temp_unzip_dir/nome_do_arquivo.dbf"

        # Passo 3: Leitura do arquivo .dbf e transformação em um DataFrame do Pandas
        dbf_data = []
        for record in DBF(dbf_file_path):
            dbf_data.append(record)

        dbf_df = pd.DataFrame(dbf_data)

        # Passo 4: Realize a operação de JOIN entre DataFrames
        # Suponha que a coluna de JOIN seja 'coluna_comum'
        result_df = pd.merge(df, dbf_df, left_on='coluna_comum', right_on='coluna_comum', how='inner')

        # Create a window to display the table
        data_window = tk.Toplevel(root)
        data_window.title("Dados CSV e DBF")

        # Create combobox filters for each column
        column_comboboxes = []

        def apply_filters():
            filtered_df = result_df.copy()
            for col_name, combobox in column_comboboxes:
                selected_value = normalize_and_uppercase(combobox.get())
                if selected_value:
                    filtered_df = filtered_df[filtered_df[col_name].apply(normalize_and_uppercase) == selected_value]

            # Clear the existing rows in the Treeview
            for row in data_tree.get_children():
                data_tree.delete(row)

            # Repopulate the Treeview with the filtered data
            for index, row in filtered_df.iterrows():
                data_tree.insert("", "end", text=index, values=[row[i] for i in range(len(filtered_df.columns))])

        def clear_filters():
            for col_name, combobox in column_comboboxes:
                combobox.set("")  # Clear the selected value
            apply_filters()  # Apply filters to reset the table

        filter_frame = ttk.Frame(data_window)
        filter_frame.pack()
        data_tree_frame = ttk.Frame(data_window)
        data_tree_frame.pack()

        for col_name in result_df.columns:
            unique_values = [normalize_and_uppercase(value) for value in result_df[col_name].unique()]
            combobox = ttk.Combobox(filter_frame, values=unique_values)
            combobox.bind("<Return>", lambda event, col_name=col_name: apply_filters())
            combobox.grid(row=0, column=len(column_comboboxes), padx=5)
            column_comboboxes.append((col_name, combobox))

        apply_filters_button = ttk.Button(filter_frame, text="Aplicar Filtros", command=apply_filters)
        apply_filters_button.grid(row=0, column=len(column_comboboxes) + 1, padx=5)
        clear_filters_button = ttk.Button(filter_frame, text="Limpar Filtros", command=clear_filters)
        clear_filters_button.grid(row=0, column=len(column_comboboxes) + 2, padx=5)

        data_tree = ttk.Treeview(data_tree_frame)
        data_tree["columns"] = result_df.columns
        data_tree.heading("#0", text="Índice")
        for col_id, col_name in enumerate(result_df.columns):
            data_tree.heading(col_id, text=col_name)
            data_tree.column(col_id, width=100)

        for index, row in result_df.iterrows():
            data_tree.insert("", "end", text=index, values=[row[i] for i in range(len(result_df.columns))])

        data_tree.pack()
def download_csv(url):
    response = requests.get(url)
    if response.status_code == 200:
        return StringIO(response.text)
    else:
        return None

# Main window configuration
root = tk.Tk()
root.title("Aplicativo CSV")

# Create widgets
source_label = ttk.Label(root, text="URL do CSV:")
source_entry = ttk.Entry(root, width=40)
col_index_label = ttk.Label(root, text="Índice da Coluna (opcional):")
col_index_entry = ttk.Entry(root, width=5)
fetch_button = ttk.Button(root, text="Carregar e Mostrar Tabela", command=download_csv_and_show_table)

# Position widgets
source_label.grid(row=0, column=0, sticky="w")
source_entry.grid(row=0, column=1, columnspan=2, sticky="w")
col_index_label.grid(row=1, column=0, sticky="w")
col_index_entry.grid(row=1, column=1, sticky="w")
fetch_button.grid(row=1, column=2)

# Start the GUI
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_7744\1446195408.py", line 57, in download_csv_and_show_table
    with zipfile.ZipFile(response.content) as zip_ref:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\Eduarda\anaconda3\Lib\zipfile.py", line 1302, in __init__
    self._RealGetContents()
  File "d:\Users\Eduarda\anaconda3\Lib\zipfile.py", line 1365, in _RealGetContents
    endrec = _EndRecData(fp)
             ^^^^^^^^^^^^^^^
  File "d:\Users\Eduarda\anaconda3\Lib\zipfile.py", line 292, in _EndRecData
    fpin.seek(0, 2)
    ^^^^^^^^^
AttributeError: 'bytes' object has no attribute 'seek'
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*a